In [6]:
import header
from v0.models import MindtoolsSkillGroup, MindtoolsSkillSubgroup, Content
from v0.index import VectorIndex
import pandas as pd
from tqdm import tqdm
import random

In [7]:
# get 10 random content
random_pks = random.sample(list(Content.objects.all().values_list('pk', flat=True)), 10)
contents = Content.objects.filter(pk__in=random_pks)

In [8]:
# generate index
index = VectorIndex(MindtoolsSkillSubgroup.objects.all())

[v0.VectorIndex_MindtoolsSkillSubgroup] 2022-07-20 11:15:57,887 INFO MainThread [index:_generate_index:47] | Generated index with a total of 87 vectors in 0.262s


In [16]:
output = pd.DataFrame(columns=['content_title', 'content_url', 'MindtoolsSkillGroups', 'MindtoolsSkillSubgroups'])

for i, content in tqdm(enumerate(contents.iterator()), total=contents.count()):
    # get the top_k skills for the content
    results, rankings, query_vector = index.query(content.embedding_all_mpnet_base_v2, k=5)

    # perform transaction
    results: list[MindtoolsSkillSubgroup] = list(results)

    # add to output
    output.loc[i] = [content.title, content.url, ', '.join(list(set([str(x.group) for x in results]))), ', '.join([str(x) for x in results])]

output.to_csv('article_mindtools_experiment.csv', index=False)

100%|██████████| 10/10 [00:02<00:00,  3.81it/s]
